In [1]:
using DynamicHMCModels

ProjDir = rel_path_d("..", "scripts", "12")

df = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(df) # Should be 10x5

(10, 5)

New col logpop, set log() for population data

In [2]:
df[:logpop] = map((x) -> log(x), df[:population]);
df[:society] = 1:10;

first(df[[:total_tools, :logpop, :society]], 5)

struct m_12_06d_model{TY <: AbstractVector, TX <: AbstractMatrix,
  TS <: AbstractVector}
    "Observations (total_tools)."
    y::TY
    "Covariates (logpop)"
    X::TX
    "Society"
    S::TS
    "Number of observations (10)"
    N::Int
    "Number of societies (also 10)"
    N_societies::Int
end

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::m_12_06d_model)(θ)
    @unpack y, X, S, N, N_societies = problem   # extract the data
    @unpack β, α, σ = θ  # β : a, bp, α : a_society
    ll = 0.0
    ll += logpdf(Cauchy(0, 1), σ)
    ll += sum(logpdf.(Normal(0, σ), α)) # α[1:10]
    ll += logpdf.(Normal(0, 10), β[1]) # a
    ll += logpdf.(Normal(0, 1), β[2]) # a
    ll += sum(
      [loglikelihood(Poisson(exp(α[S[i]] + dot(X[i, :], β))), [y[i]]) for i in 1:N]
    )
    ll
end

Instantiate the model with data and inits.

In [4]:
N = size(df, 1)
N_societies = length(unique(df[:society]))
X = hcat(ones(Int64, N), df[:logpop]);
S = df[:society]
y = df[:total_tools]
p = m_12_06d_model(y, X, S, N, N_societies);
θ = (β = [1.0, 0.25], α = rand(Normal(0, 1), N_societies), σ = 0.2)
p(θ)

-256.66968436926805

Write a function to return properly dimensioned transformation.

In [5]:
problem_transformation(p::m_12_06d_model) =
    as( (β = as(Array, size(p.X, 2)), α = as(Array, p.N_societies), σ = asℝ₊) )
# Wrap the problem with a transformation, then use Flux for the gradient.
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));
#∇P = ADgradient(:ForwardDiff, P);

Tune and sample.

In [6]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.0029 s/step ...done
MCMC, adapting ϵ (25 steps)
0.004 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0024 s/step ...done
MCMC, adapting ϵ (100 steps)
0.0011 s/step ...done
MCMC, adapting ϵ (200 steps)
0.00075 s/step ...done
MCMC, adapting ϵ (400 steps)
0.00046 s/step ...done
MCMC, adapting ϵ (50 steps)
0.00055 s/step ...done
MCMC (1000 steps)
0.00037 s/step ...done


We use the transformation to obtain the posterior from the chain.

In [7]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));
posterior[1:5]

5-element Array{NamedTuple{(:β, :α, :σ),Tuple{Array{Float64,1},Array{Float64,1},Float64}},1}:
 (β = [2.290525498121232, 0.11867225344297483], α = [-0.7372490643838894, 0.02687319171878795, -0.04471474367170464, 0.5563583049032732, -0.10389694706951819, -0.24521565323010608, 0.19187753624190235, -0.001335931646366, 0.6032544345416558, 0.44648652986226217], σ = 0.47335903460864764)  
 (β = [0.37782543216117753, 0.33359181730040227], α = [0.14601006363600166, 0.27311549375260497, 0.2619065503475652, 0.2543100557896106, 0.24058800515610357, -0.15178847646201996, 0.2489489775618811, -0.2714382519761657, 0.3620856824542829, -0.2859225217464723], σ = 0.19993810274769824)   
 (β = [2.194566795259319, 0.1516434615311157], α = [-0.5506033867466331, 0.0669777325018687, -0.2655292830413042, 0.14034743555082022, -0.06663199496424735, -0.6189157071823654, 0.17647614268012354, -0.2790137310077481, 0.19316542236736556, 0.2056268398113081], σ = 0.2983744834903703)     
 (β = [0.37598460907307973, 0.347

Extract the parameter posterior means.

In [8]:
posterior_β = mean(posterior[i].β for i in 1:length(posterior))
posterior_α = mean(posterior[i].α for i in 1:length(posterior))
posterior_σ = mean(posterior[i].σ for i in 1:length(posterior))

0.3108333881451977

Effective sample sizes (of untransformed draws)

In [9]:
ess = mapslices(effective_sample_size, get_position_matrix(chain); dims = 1)
ess

1×13 Array{Float64,2}:
 904.767  921.685  993.999  865.232  …  1000.0  947.396  896.975  507.48

NUTS-specific statistics

In [10]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.94, min/25%/median/75%/max: 0.37 0.91 0.97 0.99 1.0
  termination: AdjacentTurn => 5% DoubledTurn => 95%
  depth: 3 => 2% 4 => 97% 5 => 1%


CmdStan result

In [11]:
m_12_6_result = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                            Mean                SD               Naive SE             MCSE            ESS
            a          1.076167468  0.7704872560 0.01218247319 0.0210530022 1000.000000
           bp         0.263056273  0.0823415805 0.00130193470 0.0022645077 1000.000000
  a_society.1   -0.191723568  0.2421382537 0.00382854195 0.0060563054 1000.000000
  a_society.2    0.054569029  0.2278506876 0.00360263570 0.0051693148 1000.000000
  a_society.3   -0.035935050  0.1926364647 0.00304584994 0.0039948433 1000.000000
  a_society.4    0.334355037  0.1929971201 0.00305155241 0.0063871707  913.029080
  a_society.5    0.049747513  0.1801287716 0.00284808595 0.0043631095 1000.000000
  a_society.6   -0.311903245  0.2096126337 0.00331426674 0.0053000536 1000.000000
  a_society.7    0.148637507  0.1744680594 0.00275858223 0.0047660246 1000.000000
  a_society.8   -0.164567976  0.1821341074 0.00287979309 0.0034297298 1000.000000
  a_society.9    0.277066965  0.1758237250 0.00278001719 0.0055844175  991.286501
 a_society.10   -0.094149204  0.2846206232 0.00450024719 0.0080735022 1000.000000
sigma_society    0.310352849  0.1374834682 0.00217380450 0.0057325226  575.187461
";

Show means

In [12]:
[posterior_β, posterior_α, [posterior_σ]]

3-element Array{Array{Float64,1},1}:
 [1.093275412270956, 0.26189148634478476]                                                                                                                                                                              
 [-0.20447658075645123, 0.03995204791279477, -0.047464893116019036, 0.33165269647188084, 0.03783032952961538, -0.32838162872412746, 0.1486054339351236, -0.1689446350771479, 0.27289518671456775, -0.10641502292248989]
 [0.3108333881451977]                                                                                                                                                                                                  

End of m12.6d.jl

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*